In [6]:
#import libraries
import os
import datetime as dt
from pathlib import Path
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import matplotlib.pyplot as plt

In [7]:
# Load the data into a Pandas DataFrame
df_Football_Season = pd.read_csv(
    Path("2021_2011_regular_ season_1.csv",
                          index_col=["Years","Teams","W"],
                          parse_dates = True, 
                          infer_datetime_format = True)
)

# Drop Date column
df_Football_Season=df_Football_Season.drop(columns=["Date"])

# Display sample data
df_Football_Season.head(10)

,Years,Teams,W,L,T,PCT,PF,PA,Net Pts,Home,Road,Div,Pct,Conf,Pct.1,Non-Conf,Strk,Last 5
0,2021,Bills,11,6,0,0.647,483,289,194,6 - 3 - 0,5 - 3 - 0,5 - 1 - 0,0.833,7 - 5 - 0,0.583,4 - 1 - 0,4W,4 - 1 - 0
1,2021,Dolphins,9,8,0,0.529,341,373,-32,6 - 3 - 0,3 - 5 - 0,4 - 2 - 0,0.667,6 - 6 - 0,0.500,3 - 2 - 0,1W,4 - 1 - 0
2,2021,Patriots,10,7,0,0.588,462,303,159,4 - 5 - 0,6 - 2 - 0,3 - 3 - 0,0.500,8 - 4 - 0,0.667,2 - 3 - 0,1L,2 - 3 - 0
3,2021,Jets,4,13,0,0.235,310,504,-194,3 - 6 - 0,1 - 7 - 0,0 - 6 - 0,0.000,4 - 8 - 0,0.333,0 - 5 - 0,2L,1 - 4 - 0
4,2021,Bengals,10,7,0,0.588,460,376,84,5 - 4 - 0,5 - 3 - 0,4 - 2 - 0,0.667,8 - 4 - 0,0.667,2 - 3 - 0,1L,3 - 2 - 0
5,2021,Steelers,9,7,1,0.559,343,398,-55,6 - 2 - 1,3 - 5 - 0,4 - 2 - 0,0.667,7 - 5 - 0,0.583,2 - 2 - 1,2W,3 - 2 - 0
6,2021,Browns,8,9,0,0.471,349,371,-22,6 - 3 - 0,2 - 6 - 0,3 - 3 - 0,0.500,5 - 7 - 0,0.417,3 - 2 - 0,1W,2 - 3 - 0
7,2021,Ravens,8,9,0,0.471,387,392,-5,5 - 4 - 0,3 - 5 - 0,1 - 5 - 0,0.167,5 - 7 - 0,0.417,3 - 2 - 0,6L,0 - 5 - 0
8,2021,Titans,12,5,0,0.706,419,354,65,7 - 2 - 0,5 - 3 - 0,5 - 1 - 0,0.833,8 - 4 - 0,0.667,4 - 1 - 0,3W,4 - 1 - 0
9,2021,Colts,9,8,0,0.529,451,365,86,4 - 5 - 0,5 - 3 - 0,3 - 3 - 0,0.500,7 - 5 - 0,0.583,2 - 3 - 0,2L,3 - 2 - 0


In [8]:
# Define a dictionary containing Students data
data = {'Teams': ['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns', 'Buccaneers','Cardinals','Chargers','Chiefs','Colts','Cowboys','Dolphins','Eagles','Falcons',
        'Giants','Jaguars','Jets','Lions','Packers','Panthers','Patriots','Raiders','Rams','Ravens','Redskins','Saints','Seahawks','Steelers','Texans','Titans',
        'Vikings'],
        'W': [90,79,87,91,97,56,73,89,84,112,92,98,82,90,87,70,47,63,74,118,86,128,73,88,105,70,110,112,111,84,86,90],
        'Net Pts': [127,-258,72,106,284,-933,-347,-66,133,643,-18,380,-492,208,21,-556,-1203,-1039,-342,708,-32,1599,-1030,-43,793,-676,813,946,574,-193,-284,105],      
        'Unique Identifier': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]
}

# Convert the dictionary into DataFrame
df_SuperBowlWinners_Deep = pd.DataFrame.from_dict(data)

# Declare a list that is to be converted into a column
SuperBowlWins = [0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0]

# Using 'Previous Super Bowl Winner' as the column name
# and equating it to the list
df_SuperBowlWinners_Deep['Previous Super Bowl Winner'] = SuperBowlWins

# Set Index & DType
# df_SuperBowlWinners_Deep["Teams"] = df_SuperBowlWinners_Deep["Teams"].astype(float)
df_SuperBowlWinners_Deep.reset_index(drop=True)


# Observe the result
df_SuperBowlWinners_Deep

,Teams,W,Net Pts,Unique Identifier,Previous Super Bowl Winner
0,49ers,90,127,1,0
1,Bears,79,-258,2,0
2,Bengals,87,72,3,0
3,Bills,91,106,4,0
4,Broncos,97,284,5,1
5,Browns,56,-933,6,0
6,Buccaneers,73,-347,7,1
7,Cardinals,89,-66,8,0
8,Chargers,84,133,9,0
9,Chiefs,112,643,10,1


In [11]:
# Create a list of categorical variables 
categorical_variables = ['Unique Identifier', 'Teams']

# Display the categorical variables list
categorical_variables

['Unique Identifier', 'Teams']

In [13]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [14]:
# Create a OneHotEncoder instance (Takes non-numerical data into 0 or 1)
enc =  OneHotEncoder(sparse=False)

In [15]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(df_SuperBowlWinners_Deep[categorical_variables])
encoded_data

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names_out(categorical_variables)
)

In [18]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables_df = df_SuperBowlWinners_Deep.drop(columns = categorical_variables)

# Review the DataFrame
numerical_variables_df.head()

,W,Net Pts,Previous Super Bowl Winner
0,90,127,0
1,79,-258,0
2,87,72,0
3,91,106,0
4,97,284,1


In [19]:
# # Using the Pandas concat function, combine the DataFrames the contain the encoded categorical data and the numerical data
# encoded2_df = pd.concat(
#     [
#         numerical_variables_df,
#         encoded_df
#     ],
#     axis=1
# )


In [25]:
# Define the target set y using the 'Previous Super Bowl Winner' column
y = encoded2_df["Previous Super Bowl Winner"]

# Display a sample of y
y[:10]

0    0
1    0
2    0
3    0
4    1
5    0
6    1
7    0
8    0
9    1
Name: Previous Super Bowl Winner, dtype: int64

In [26]:
# # Define features set X by selecting all columns but IS_SUCCESSFUL
# X = encoded2_df.drop(columns=["Previous Super Bowl Winner"])

# # Review the features DataFrame
# X.head()

,W,Net Pts,Unique Identifier_1,Unique Identifier_2,Unique Identifier_3,Unique Identifier_4,Unique Identifier_5,Unique Identifier_6,Unique Identifier_7,Unique Identifier_8,...,Teams_Raiders,Teams_Rams,Teams_Ravens,Teams_Redskins,Teams_Saints,Teams_Seahawks,Teams_Steelers,Teams_Texans,Teams_Titans,Teams_Vikings
0,90,127,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,79,-258,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,87,72,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,91,106,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,97,284,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.model_selection import train_test_split

In [31]:
import random

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [89]:
X = df_SuperBowlWinners_Deep[["W", "Net Pts", "Unique Identifier", "Previous Super Bowl Winner"]].set_index("Unique Identifier")

In [90]:
X.head()

,W,Net Pts,Previous Super Bowl Winner
Unique Identifier,,,
1,90,127,0
2,79,-258,0
3,87,72,0
4,91,106,0
5,97,284,1


In [91]:
Xname = ["W", "Net Pts", "Previous Super Bowl Winner"]
yname = ["Previous Super Bowl Winner"]
df_Xname = X[Xname]
df_yname = X[yname]

In [92]:
import numpy as np

In [93]:
np_Xname = np.array(df_Xname)
np_yname = np.array(df_yname)

In [94]:
Uni_ID = df_SuperBowlWinners_Deep[["W", "Net Pts", "Previous Super Bowl Winner"]]

In [95]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
compare_df = df_SuperBowlWinners_Deep.copy()
for i in range(0,10):
    n= random.randint(0,100)
    X_train, X_test, y_train, y_test = train_test_split(np_Xname, np_yname, test_size=(0.4) , random_state=n)
    reg = RandomForestRegressor(random_state=n)
    reg.fit(X_train, y_train)
    predictions = reg.predict(Uni_ID)
    compare_df[f'predict_{n}'] = predictions

C:\Users\Chiny\AppData\Local\Temp\ipykernel_5984\2355038017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg.fit(X_train, y_train)
C:\Users\Chiny\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\Chiny\AppData\Local\Temp\ipykernel_5984\2355038017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg.fit(X_train, y_train)
C:\Users\Chiny\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\Chiny\AppData\Local\Temp\ipykernel_5984\2355038017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please

In [96]:
compare_df

,Teams,W,Net Pts,Unique Identifier,Previous Super Bowl Winner,predict_13,predict_82,predict_60,predict_18,predict_31,predict_61,predict_16,predict_62,predict_92,predict_71
0,49ers,90,127,1,0,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Bears,79,-258,2,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Bengals,87,72,3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Bills,91,106,4,0,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Broncos,97,284,5,1,0.98,0.98,0.99,0.92,0.99,1.00,0.97,0.92,0.99,0.99
5,Browns,56,-933,6,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Buccaneers,73,-347,7,1,0.95,0.93,0.99,0.89,0.97,0.99,0.88,0.89,0.99,0.98
7,Cardinals,89,-66,8,0,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,Chargers,84,133,9,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Chiefs,112,643,10,1,0.99,1.00,0.99,0.98,0.99,1.00,0.99,1.00,1.00,1.00


In [82]:
# # Configuring a Monte Carlo simulation to forecast 252 days cumulative returns
# MC_252Days = MCSimulation(
#     portfolio_data = numerical_variables_df,
#     num_simulation = 500,
#     num_trading_days = 17
# )